In [1]:
import random
import numpy as np
from copy import deepcopy
random.seed(1)

In [2]:
# Hyperparameters
alpha = 0.2             # step size
gamma = 0.6             # discount factor
epsilon = 0.1             # prob epsilon of choosing random action (0 == pure exploitation, 1 == pure exploration)
n = 5                   # grid size
G = 3                   # gold limit
time = 40               # time limit (a.k.a action limit)
episodes = 10000          # number of episodes to simulate for
num_actions = 4         # number of actions
num_states = 23         # number of states

In [3]:
mine = "m"
sue = "s"
house = "h"
empty = "*"

map = [
    [house, empty, empty, empty, empty],
    [empty, empty, empty, empty, empty],
    [empty, empty, empty, empty, empty],
    [empty, empty, empty, empty, empty],
    [sue, empty, empty, empty, mine]
]

for row in map:
    print(' '.join(row))



h * * * *
* * * * *
* * * * *
* * * * *
s * * * m


In [4]:
class State:
    def __init__(self, map, sue_pos, sue_gold):
        self.map = map
        self.sue_pos = sue_pos
        self.sue_gold = sue_gold
        

In [5]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

ACTIONS = [UP, DOWN, LEFT, RIGHT]

In [6]:
start_state = State(map=map, sue_pos=[4, 0], sue_gold=0)

In [7]:
def sue_new_pos(state, action):
    going_into_house = False
    going_into_mine = False
    going_out_of_bounds = False
    p = deepcopy(state.sue_pos)
    if action == UP:
        if p == [1, 0]:
            going_into_house = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        elif p[0] == 0:
            going_out_of_bounds = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        else:
            p[0] = max(0, p[0] - 1)
    elif action == DOWN:
        if p == [3, 4]:
            going_into_mine = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        elif p[0] == 4:
            going_out_of_bounds = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        else:
            p[0] = min(len(state.map) - 1, p[0] + 1)
    elif action == LEFT:
        if p == [0, 1]:
            going_into_house = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        elif p[1] == 0:
            going_out_of_bounds = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        else:
            p[1] = max(0, p[1] - 1)
    elif action == RIGHT:
        if p == [4, 3]:
            going_into_mine = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        elif p[1] == 4:
            going_out_of_bounds = True
            return p, going_into_house, going_into_mine, going_out_of_bounds
        else:
            p[1] = min(len(state.map) - 1, p[1] + 1)
    else:
        raise ValueError(f"Unknown action {action}")

    return p, going_into_house, going_into_mine, going_out_of_bounds


def act(state, action):
            
    p, into_house, into_mine, out_of_bounds = sue_new_pos(state, action)
    gold = state.sue_gold
    map_item = state.map[p[0]][p[1]]
    new_map = deepcopy(state.map)

    if map_item == empty:                   # moving onto empty block
        old = state.sue_pos
        new_map[old[0]][old[1]] = empty
        new_map[p[0]][p[1]] = sue
        # print(f"Moving onto empty block")
        reward = 0
    else:
        if into_mine == True:               # moving into mine
            if gold < G:
                gold += 1
                # print(f"Going into mine... gold increased by one to {gold}")
            else:
                # print(f"Going into mine... gold is at max, {gold}")
                pass
            reward = 0
        elif into_house == True:            # moving into house
            if gold > 0:
                # print(f"Going into house with {gold} gold! Yay")
                reward = gold
                gold = 0
            else:
                # print(f"Going into house with no gold :(")
                reward = 0
        elif out_of_bounds == True:         # going out of bounds
            # print(f"Sue trying to go out of bounds! Bad Sue!")
            reward = 0
        else:
            raise ValueError(f"Unknown map item {map_item}")
    
    return State(map=new_map, sue_pos=p, sue_gold=gold), reward

In [8]:
q_table = {}
for x in range(0, n):
    for y in range(0, n):
        if x == 0 and y == 0:           # location of house
            pass
        elif x == 4 and y == 4:         # location of mine
            pass
        else:
            for a in range(G+1):     # amount of gold Sue has
                q_table[x, y, a] = np.random.normal(0, 1, (len(ACTIONS)))

In [9]:
def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        return random.choice(ACTIONS) 
    else:
        return np.argmax(q_table[(state.sue_pos[0], state.sue_pos[1], state.sue_gold)])

In [ ]:
rewards = []
for e in range(episodes):
    
    state = start_state
    total_reward = 0
    sequence = ""
    
    for _ in range(time):
        action = choose_action(state)
        if action == 0:
            sequence += "U"
        elif action == 1:
            sequence += "D"
        elif action == 2:
            sequence += "L"
        elif action == 3:
            sequence += "R"

        next_state, reward = act(state, action)
        total_reward += reward

        x = state.sue_pos[0]
        x2 = next_state.sue_pos[0]

        y = state.sue_pos[1]
        y2 = next_state.sue_pos[1]

        a = state.sue_gold
        a2 = next_state.sue_gold

        q_table[(x, y, a)][action] = (1-alpha)*q_table[(x, y, a)][action] + alpha*(reward + gamma*np.max(q_table[(x2, y2, a2)]))
        # for row in state.map:
        #     print(' '.join(row))
        state = next_state

    print(f"Episode {e + 1}: total reward -> {total_reward}")
    print(f"Sequence: {sequence}")
    rewards.append(total_reward)